Interesting References:

https://en.wikipedia.org/wiki/Multiclass_classification

https://en.wikipedia.org/wiki/Tf%E2%80%93idf

http://scikit-learn.org/stable/modules/multiclass.html#multiclass

https://www.kaggle.com/viznrvn/optimal-parameters-for-svc-using-gridsearch

http://scikit-learn.org/0.15/auto_examples/grid_search_digits.html

https://datascienceplus.com/multi-class-text-classification-with-scikit-learn/


**Basic data Understanding**

This is a first look into data, to understand the database and how it works. Showing the most Frequent occurences and basic plots.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import operator
from sklearn.preprocessing import label_binarize
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
print(os.listdir("../input"))

In [ ]:
#Load the Training Data
import json
with open('../input/train.json') as f:
    data = json.load(f)
#Treat Data with TfidfVectorizer
train_df = pd.DataFrame(data)
final=[]
for el in train_df['ingredients']:
    test= ' '.join(el).lower()
    final.append(test)
vectorizer = TfidfVectorizer(binary = True)
X = vectorizer.fit_transform(final)
#Load the Test Data and do the same treatment as Train Data
with open('../input/test.json') as f:
    data_test = json.load(f)
test_df = pd.DataFrame(data_test)
final_test=[]
for el in test_df['ingredients']:
    test= ' '.join(el).lower()
    final_test.append(test)
X_test = vectorizer.transform(final_test)

In [ ]:
# convert dataset to Train and Validation Sets
from sklearn.model_selection import train_test_split
y=train_df['cuisine']
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2)

In [ ]:
#set(list(y_validation))

**MODELS  TESTING**

Creating Diffent model to evaluate

In [ ]:
import sklearn
from sklearn import metrics
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

models=[];accs=[];times=[]
def running(mod):
    mod_name=mod[:-2]
    print(mod_name)
    if mod_name not in models:
        models.append(mod_name)
        start=time.time()
        #print ('================ Creating Models ================')
        model=eval(mod)
        #print('================ Fitting Models ================')
        model.fit(X_train,y_train)
        #print('================ Predictions ================')
        y_model=model.predict(X_validation)
        end=time.time()
        #print("The Model achieve Accuracy:",metrics.accuracy_score(y_test, y_model), 'in', end-start,'sec')
        accs.append(metrics.accuracy_score(y_validation, y_model))
        times.append(end-start)
lista=['BernoulliNB()',
       'DecisionTreeClassifier()',
       'ExtraTreeClassifier()',
       'ExtraTreesClassifier()',
       'LogisticRegression()',
       'LinearSVC()',
       'RidgeClassifier()',
       'RandomForestClassifier()',
       'NearestCentroid()']
       #'MLPClassifier(max_iter=500)']
rrun=False
if rrun==True:
    for el in lista:
        running(el)
    for_df = [('Model', models),
             ('Accuracy', accs),
             ('Time', times)]
    df_models = pd.DataFrame.from_items(for_df)
    print (df_models)

In [ ]:
from sklearn.linear_model import SGDClassifier
rrun=True
if rrun==True:
    start=time.time()
    sdg = SGDClassifier().fit(X_train,y_train).predict(X_validation)
    end=time.time()
    print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, sdg), 'in', end-start,'sec')
#Result:
#The Model achieve Accuracy: 0.7808925204274042 in 0.6468749046325684 sec

In [ ]:
from xgboost import XGBClassifier
rrun=False
if rrun==True:
    start=time.time()
    test = XGBClassifier().fit(X_train,y_train).predict(X_validation)
    end=time.time()
    print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, test), 'in', end-start,'sec')
#Result:
#The Model achieve Accuracy: 0.7314896291640478 in 150.10474228858948 sec

In [ ]:
rrun=False
if rrun==True:
    start=time.time()
    mod=LinearSVC()
    linsvc = mod.fit(X_train,y_train).predict(X_validation)
    end=time.time()
    print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, linsvc), 'in', end-start,'sec')
#Result:
#The Model achieve Accuracy: 0.7890634820867379 in 2.173555850982666 sec

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
rrun=False
if rrun==True:
    clf1 = LogisticRegression()
    clf2 = LinearSVC()
    clf3 = SGDClassifier()
    eclf = VotingClassifier(estimators=[('lr', clf1), ('LinearSVC', clf2), ('SGD', clf3)], voting='hard')
    for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'LinearSVC', 'SGDClassifier', 'Ensemble']):
        scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    

 Once LinearSVC achieve great results, I will try GridSearchCV from sklearn to find the optimal parameters for C, gamma and kernel from a given set of values to improve SVC acuracy.

In [ ]:
from sklearn.svm import SVC
print('  SVC:',SVC().get_params().keys())

In [ ]:
from sklearn.grid_search import GridSearchCV
rrun=False
if rrun==True:
    #Snipet From Optimal Parameters Search for SVC Kernel in References
    from sklearn.grid_search import GridSearchCV
    param_grid = {'C':[1,50,75,100],'gamma':[1], 'kernel':['rbf']}
    grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)
    grid.fit(X_train,y_train)
    grid.best_params_
    predic = grid.predict(X_validation)
    print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, predic))
#RESULT:
#[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 29.6min finished
#{'C': 50, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn import metrics

#According sklean help the SVC is handled according to a one-vs-one scheme. Let't try one vs All.
#   {'C': 50, 'gamma': 1, 'kernel': 'rbf'}
rrun=True
if rrun==True:
    start=time.time()
    svc=SVC(C=50,gamma=1,kernel='rbf')
    onevsall=OneVsRestClassifier(svc).fit(X_train,y_train).predict(X_validation)
    end=time.time()
    print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, onevsall), 'in', end-start,'sec')
    #Confusion Matrix
    conf_mat = confusion_matrix(y_validation, onevsall)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_mat, annot=True, fmt='d',xticklabels=set(list(y_validation)), yticklabels=set(list(y_validation)))
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
    print(metrics.classification_report(y_validation, onevsall, target_names=set(list(y_validation))))

In [ ]:
from sklearn.linear_model import LogisticRegression
rrun=True
if rrun==True:
    start=time.time()
    logreg=LogisticRegression().fit(X_train,y_train).predict(X_validation)
    end=time.time()
    print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, logreg), 'in', end-start,'sec')
    conf_mat = confusion_matrix(y_validation, logreg)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_mat, annot=True, fmt='d',xticklabels=set(list(y_validation)), yticklabels=set(list(y_validation)))
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
    print(metrics.classification_report(y_validation, logreg, target_names=set(list(y_validation))))

In [ ]:
y_val_ch=[]
for el in y_validation:
    if el=='russian':
        y_val_ch.append(1)
    else:
        y_val_ch.append(0)
y_train_ch=[]
for el in y_train:
    if el=='russian':
        y_train_ch.append(1)
    else:
        y_train_ch.append(0)
from sklearn.linear_model import RidgeClassifier
ridge_ch=RidgeClassifier().fit(X_train,y_train_ch).predict(X_validation)
print(metrics.classification_report(y_val_ch, ridge_ch))

In [ ]:
y_val_vt=[]
for el in y_validation:
    if el=='thai':
        y_val_vt.append(1)
    else:
        y_val_vt.append(0)
y_train_vt=[]
for el in y_train:
    if el=='thai':
        y_train_vt.append(1)
    else:
        y_train_vt.append(0)
ridge_vt=RidgeClassifier().fit(X_train,y_train_vt).predict(X_validation)
print(metrics.classification_report(y_val_vt, ridge_vt))

In [ ]:
y_val_gr=[]
for el in y_validation:
    if el=='moroccan':
        y_val_gr.append(1)
    else:
        y_val_gr.append(0)
y_train_gr=[]
for el in y_train:
    if el=='moroccan':
        y_train_gr.append(1)
    else:
        y_train_gr.append(0)
ridge_gr=RidgeClassifier().fit(X_train,y_train_gr).predict(X_validation)
print(metrics.classification_report(y_val_gr, ridge_gr))

In [ ]:
final_res=[]
for i,j,ai,k,l,m in zip(ridge_ch,ridge_vt,ridge_gr,onevsall,logreg,sdg):
    if i==1 and j==0 and ai==0:
        if k=='southern_us' or k=='indian':          
            res='russian'
    elif j==1 and i==0 and ai==0:
        if k=='italian' or k=='vietnamese' or k=='british':
            res='thai'
    elif j==0 and i==0 and ai==1:
        if k=='southern_us' or k=='indian' or k=='filipino' or k=='cajun_creole':
            res='moroccan'
    else:
        res=k
    final_res.append(res)
print("The Model achieve Accuracy:",metrics.accuracy_score(y_validation, final_res))
print(metrics.classification_report(y_validation, final_res, target_names=set(list(y_validation))))

In [ ]:
#final_sub=OneVsRestClassifier(svc).fit(X,y).predict(X_test)
#ids=test_df['id'] 
#my_submission = pd.DataFrame({'id': ids, 'cuisine': final_sub})
#my_submission.to_csv('submission.csv', index=False)
#print (" ======================= End ======================= ")